In [11]:
import random 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import operator
from tqdm import tqdm

class KNN_Classifier:
    def __init__(self, K=1):
        self.n_neighbor = K
    
    def fit(self, X, Y):
        self.train_data = X
        self.train_labels = Y
    
    def get_distance(self, train_X, test_X):
        total = 0
        for i in range(len(train_X)):
            total = total + np.square(test_X[i]-train_X[i])
        return np.sqrt(total)
    
    def predict_labels(self, test_X):
        test_labels=[]
        for test_d in test_X:
            pred_label = self.predict_label(test_d)
            test_labels.append(pred_label)
        return test_labels
    
    def predict_label(self, test_x):
        distance_map = {}
        for i in range(len(self.train_data)):
            dist = self.get_distance(test_x, self.train_data[i])
            distance_map[i]=dist
            
        sorted_dist = sorted(distance_map.items(), key = operator.itemgetter(1))

        neighbors_list = []
        for i in range(self.n_neighbor):
            neighbors_list.append(sorted_dist[i][0])
        

        majority_vote = {}
        for i in range(len(neighbors_list)):
            label = self.train_labels[neighbors_list[i]]

            if label in majority_vote:
                majority_vote[label]+=1
            else:
                majority_vote[label]=1
        

        sorted_vote = sorted(majority_vote.items(), key=operator.itemgetter(1), reverse=True)
        pred_label = sorted_vote[0][0]
        
        return pred_label
    
def accuracy_score(actual_labels, pred_labels):
    correct = 0
    for i in range(len(actual_labels)):
        if actual_labels[i] == pred_labels[i]:
            correct += 1
    accuracy = correct/len(pred_labels)
    return accuracy

In [18]:
#features = train_df.columns.to_list()[1:]
def forward_selection(train_df, test_df, model_accuracy_cache):
    features = get_features_list(train_df)
    selected_features = []
    K = 3
    all_best_accuracy = 0.0
    all_best_features = []
    num_features = []  # list to keep track of the number of features
    accuracies = []  # list to keep track of the accuracies
    print("Starting Forward Selection Search")
    prev_acc = 0.0
    for i in tqdm(range(len(features))):
        print("Level ", i+1, " using ", i+1, " features: ")
        best_accuracy = 0.0 
        best_feature = None
        for feature in features:
            if feature not in selected_features:
                current_features = selected_features + [feature]
                current_features = sorted(current_features)
                key = tuple(current_features)
                if key not in model_accuracy_cache:
                    train_X, train_Y, test_X, test_Y = get_train_test_data_for_selected_features(train_df, test_df, current_features)

                    model = KNN_Classifier(K=K)
                    model.fit(train_X, train_Y)
                    predictions = model.predict_labels(test_X)

                    accuracy = accuracy_score(test_Y, predictions)
                    model_accuracy_cache[key] = accuracy
                else:
                    accuracy = model_accuracy_cache[key]
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_feature = feature

                print("\tAccuracy: ", accuracy, " after adding ", feature)
                #print("Best feature list1: ", all_best_features)

        if best_feature is not None:  # Only add feature if one improved the model
            selected_features.append(best_feature)
            features.remove(best_feature)  # Remove the selected feature from the original list
            num_features.append(len(selected_features))  # append the current number of features
            accuracies.append(best_accuracy)  # append the current best accuracy

        if best_accuracy > all_best_accuracy:
            all_best_accuracy = best_accuracy
            all_best_features = selected_features.copy()
            print("Best feature: ", best_feature, " with accuracy: ", best_accuracy)
            print("Feature list: ", selected_features)
            #print("Best feature list3: ", all_best_features)
        else:
            print("Attention!! Accuracy is decreasing!")
            print("Best feature: ", best_feature, " with accuracy: ", best_accuracy)
            print("Feature list: ", selected_features)
            #print("Best feature list4: ", all_best_features)
        print()

    all_best_features = sorted(all_best_features)
    print(f'Selected features: {all_best_features}')
    print("Best accuracy: ", all_best_accuracy)
    
    return num_features, accuracies, all_best_features, model_accuracy_cache
    
def backward_selection(train_df, test_df, model_accuracy_cache):
    features = get_features_list(train_df)
    selected_features = features  # start with all features
    K = 3
    all_best_accuracy = 0.0
    all_best_features = selected_features.copy()

    # First evaluate the model with all features
    train_X, train_Y, test_X, test_Y = get_train_test_data_for_selected_features(train_df, test_df, selected_features)
    model = KNN_Classifier(K=K)
    model.fit(train_X, train_Y)
    predictions = model.predict_labels(test_X)
    all_best_accuracy = accuracy_score(test_Y, predictions)
    print("Initial accuracy with all features: ", all_best_accuracy)
    
    num_features = []  # list to keep track of the number of features
    accuracies = []  # list to keep track of the accuracies
    print("Starting Backward Selection Search")
    for i in tqdm(range(len(features))):
        print("Level ", len(features)-i-1, " using ", len(features)-i-1, " features: ")
        best_accuracy = 0.0 
        best_feature = None
        for feature in selected_features:
            current_features = selected_features.copy()
            current_features.remove(feature)
            current_features = sorted(current_features)
            key = tuple(current_features)
            if key not in model_accuracy_cache:
                train_X, train_Y, test_X, test_Y = get_train_test_data_for_selected_features(train_df, test_df, current_features)

                model = KNN_Classifier(K=K)
                model.fit(train_X, train_Y)
                predictions = model.predict_labels(test_X)

                accuracy = accuracy_score(test_Y, predictions)
                model_accuracy_cache[key] = accuracy
            else:
                accuracy = model_accuracy_cache[key]

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_feature = feature
            
            print("\tAccuracy: ", accuracy, " after removing ", feature)

        if best_feature is not None:  # Only remove feature if it improved the model
            selected_features.remove(best_feature)  # Remove the selected feature from the original list
            num_features.append(len(selected_features))  # append the current number of features
            accuracies.append(best_accuracy)  # append the current best accuracy

        if best_accuracy >= all_best_accuracy:
            all_best_accuracy = best_accuracy
            all_best_features = selected_features.copy()
            print("Removed feature: ", best_feature, " with accuracy: ", best_accuracy)
            print("Current feature list: ", selected_features)
        else:
            print("Attention!! Accuracy is decreasing!")
            print("Removed feature: ", best_feature, " with accuracy: ", best_accuracy)
            print("Current feature list: ", selected_features)
        print()

    all_best_features = sorted(all_best_features)
    print(f'Selected features: {all_best_features}')
    print("Best accuracy: ", all_best_accuracy)
    
    return num_features, accuracies, all_best_features, model_accuracy_cache

In [19]:
def load_diabetes_dataset(filename = 'diabetes.csv'):
    data = pd.read_csv(filename)
    
    # Save original column names
    original_columns = data.columns.tolist()

    # Prepare new column names
    new_cols = ['label' if col=='label' else i+1 for i, col in enumerate(data.columns)]

    # Rename columns
    data.columns = new_cols

    # Create a dictionary mapping new column names to old ones
    col_name_mapping = {new: old for new, old in zip(new_cols, original_columns)}

    return data, col_name_mapping

def min_max_scaler(train, test):
    min_val = train.min()
    max_val = train.max()

    train_normalized = (train - min_val) / (max_val - min_val)
    test_normalized = (test - min_val) / (max_val - min_val)

    return train_normalized, test_normalized

def train_test_split(data, test_frac=0.25):
    test_df = data.sample(frac = test_frac, random_state=42)
    train_df = data.drop(test_df.index)

    test_df = test_df.reset_index(drop=True)
    train_df = train_df.reset_index(drop=True)
    
    print("Data size: ", data.shape)
    print("Train size: ", train_df.shape)
    print("Test size: ", test_df.shape)

    train_df, test_df = min_max_scaler(train_df, test_df)
    
    return train_df, test_df

def get_train_test_data_for_selected_features(train_df, test_df, features_to_be_selected):
    
    train_data = train_df[features_to_be_selected].values
    train_labels = train_df['label'].values

    test_data = test_df[features_to_be_selected].values
    test_labels = test_df['label'].values
    
    return train_data, train_labels, test_data, test_labels

def get_features_list(data):
    columns = list(data.columns)
    features = []
    for val in columns:
        if val != 'label':
            features.append(val)
    return features

def plot_feature_count_vs_accuracies(num_features, accuracies, title):
    plt.plot(num_features, accuracies)
    plt.xlabel('Number of Features')
    plt.ylabel('Accuracy')
    plt.title(title)
    plt.show()

In [20]:
def load_dataset(file_name="CS170_small_Data__18.txt"):
    if file_name == "diabetes":
        return load_diabetes_dataset()
    # Determine the number of columns in your data file if not known
    with open(file_name, 'r') as f:
        line = f.readline()
        num_cols = len(line.split())

    # Create column names
    col_names = ['label'] + [i+1 for i in range(num_cols - 1)]

    # Load the data
    data = pd.read_csv(file_name, names=col_names, delim_whitespace=True)
    
    if file_name == "CS170_XXXlarge_Data__12.txt":
        print("Sampling 50% of dataset!!")
        data = data.sample(frac=0.5, random_state=42).reset_index(drop=True)

    return data, col_names


In [109]:
# data = load_dataset()
# data.head(3)

In [68]:
model_accuracy_cache = {}

data, col_name_mapping = load_diabetes_dataset()
test_frac=0.25
train_df, test_df = train_test_split(data, test_frac)
print("Data size: ", data.shape)
print("Train size: ", train_df.shape)
print("Test size: ", test_df.shape)

Data size:  (768, 9)
Train size:  (576, 9)
Test size:  (192, 9)


In [69]:
#data.head(3)

In [70]:
features = get_features_list(train_df)
train_X, train_Y, test_X, test_Y = get_train_test_data_for_selected_features(train_df, test_df,features)

model = KNN_Classifier(K=3)
model.fit(train_X, train_Y)
predictions = model.predict_labels(test_X)

accuracy = accuracy_score(test_Y, predictions)
accuracy

selected_features:  [1, 2, 3, 4, 5, 6, 7, 8]


0.703125

In [91]:
# num_features1, accuracies1, all_best_features1, model_accuracy_cache = forward_selection(train_df, test_df, 
#                                                                                       model_accuracy_cache)

In [92]:
# num_features2, accuracies2, all_best_features2, model_accuracy_cache = backward_selection(train_df, test_df, 
#                                                                                        model_accuracy_cache)

In [93]:
# title = 'Forward Selection: Accuracy vs Number of Features'
# plot_feature_count_vs_accuracies(num_features1, accuracies1, title)

# title = 'Backward Selection: Accuracy vs Number of Features'
# plot_feature_count_vs_accuracies(num_features2, accuracies2, title)

In [87]:
selected = [col_name_mapping[feat] for feat in all_best_features2]
selected

['Glucose', 'Age']

In [21]:
def apply_feature_selection(feature_selection, file_name, accuracy_cache):
    data, _ = load_dataset(file_name)
    
    test_frac=0.25
    train_df, test_df = train_test_split(data, test_frac)


    if feature_selection == '1':
        num_features1, accuracies1, \
        all_best_features1, accuracy_cache = forward_selection(train_df,test_df,accuracy_cache)
        title = 'Forward Selection: Accuracy vs Number of Features'
        plot_feature_count_vs_accuracies(num_features1, accuracies1, title)

    elif feature_selection == '2':
        num_features1, accuracies1, \
        all_best_features1, accuracy_cache = backward_selection(train_df,test_df,accuracy_cache)
        title = 'Backward Selection: Accuracy vs Number of Features'
        plot_feature_count_vs_accuracies(num_features1, accuracies1, title)
    return num_features1, accuracies1, all_best_features1, accuracy_cache

In [ ]:
if __name__ == "__main__":
    small_data_accuracy_cache = {}
    large_data_accuracy_cache = {}
    xlarge_data_accuracy_cache = {}
    diabetes_data_accuracy_cache = {}
    while(True):
        print("Welcome to Feature Selection Algorithm!")
        data_selection = input("Select the dataset you want to test\n" + \
                                "\t 1 for Small Dataset \n" + \
                                "\t 2 for Large Dataset \n" + \
                                "\t 3 for Large Dataset \n" + \
                                "\t 4 for Diabetes Dataset")
        
        feature_selection = input("Select feature selection option \n" + \
                                 "\t 1 for Forward Feature Selection \n" + \
                                 "\t 2 for Backward Feature Selection")
        
        if data_selection == '1':
            file_name="CS170_small_Data__18.txt"
            #data = load_dataset(file_name)
            num_features1, accuracies1, all_best_features1, large_data_accuracy_cache = apply_feature_selection(
                feature_selection, file_name, large_data_accuracy_cache)
        
        elif data_selection == '2':
            file_name="CS170_large_Data__31.txt"
            num_features1, accuracies1, all_best_features1, large_data_accuracy_cache = apply_feature_selection(
                feature_selection, file_name, large_data_accuracy_cache)
        
        elif data_selection == '3':
            file_name="CS170_XXXlarge_Data__12.txt"
            num_features1, accuracies1, all_best_features1, xlarge_data_accuracy_cache = apply_feature_selection(
                feature_selection, file_name, xlarge_data_accuracy_cache)
        
        elif data_selection == '4':
            filename = 'diabetes.csv'
            num_features1, accuracies1, all_best_features1, diabetes_data_accuracy_cache = apply_feature_selection(
                feature_selection, file_name, diabetes_data_accuracy_cache)

Welcome to Feature Selection Algorithm!
Select the dataset you want to test
	 1 for Small Dataset 
	 2 for Large Dataset 
	 3 for Large Dataset 
	 4 for Diabetes Dataset3
Select feature selection option 
	 1 for Forward Feature Selection 
	 2 for Backward Feature Selection1
Sampling 50% of dataset!!
Data size:  (2000, 81)
Train size:  (1500, 81)
Test size:  (500, 81)
Starting Forward Selection Search


  0%|                                                    | 0/80 [00:00<?, ?it/s]

	Accuracy:  0.758  after adding  1
	Accuracy:  0.73  after adding  2
	Accuracy:  0.748  after adding  3
	Accuracy:  0.74  after adding  4
	Accuracy:  0.734  after adding  5
	Accuracy:  0.734  after adding  6
	Accuracy:  0.766  after adding  7
	Accuracy:  0.754  after adding  8
	Accuracy:  0.758  after adding  9
	Accuracy:  0.822  after adding  10
	Accuracy:  0.73  after adding  11
	Accuracy:  0.734  after adding  12
	Accuracy:  0.764  after adding  13
	Accuracy:  0.742  after adding  14
	Accuracy:  0.754  after adding  15
	Accuracy:  0.75  after adding  16
	Accuracy:  0.736  after adding  17
	Accuracy:  0.74  after adding  18
	Accuracy:  0.74  after adding  19
	Accuracy:  0.728  after adding  20
	Accuracy:  0.756  after adding  21
	Accuracy:  0.722  after adding  22
	Accuracy:  0.748  after adding  23
	Accuracy:  0.76  after adding  24
	Accuracy:  0.748  after adding  25
	Accuracy:  0.772  after adding  26
	Accuracy:  0.754  after adding  27
	Accuracy:  0.756  after adding  28
	Accurac

  1%|▌                                        | 1/80 [02:43<3:35:05, 163.36s/it]

	Accuracy:  0.75  after adding  80
Best feature:  10  with accuracy:  0.822
Feature list:  [10]

	Accuracy:  0.806  after adding  1
	Accuracy:  0.808  after adding  2
	Accuracy:  0.788  after adding  3
	Accuracy:  0.814  after adding  4
	Accuracy:  0.806  after adding  5
	Accuracy:  0.82  after adding  6
	Accuracy:  0.804  after adding  7
	Accuracy:  0.822  after adding  8
	Accuracy:  0.812  after adding  9
	Accuracy:  0.794  after adding  11
	Accuracy:  0.824  after adding  12
	Accuracy:  0.832  after adding  13
	Accuracy:  0.832  after adding  14
	Accuracy:  0.808  after adding  15
	Accuracy:  0.84  after adding  16
	Accuracy:  0.792  after adding  17
	Accuracy:  0.824  after adding  18
	Accuracy:  0.81  after adding  19
	Accuracy:  0.832  after adding  20
	Accuracy:  0.814  after adding  21
	Accuracy:  0.818  after adding  22
	Accuracy:  0.796  after adding  23
	Accuracy:  0.802  after adding  24
	Accuracy:  0.86  after adding  25
	Accuracy:  0.826  after adding  26
	Accuracy:  0.82

...

	Accuracy:  0.792  after adding  79
Attention!! Accuracy is decreasing!
Best feature:  5  with accuracy:  0.798
Feature list:  [10, 63, 71, 13, 41, 25, 22, 1, 8, 19, 77, 24, 45, 6, 27, 39, 49, 75, 66, 15, 38, 74, 21, 51, 56, 7, 50, 55, 80, 4, 37, 5]

	Accuracy:  0.772  after adding  2
	Accuracy:  0.778  after adding  3
	Accuracy:  0.79  after adding  9
	Accuracy:  0.782  after adding  11
	Accuracy:  0.77  after adding  12
	Accuracy:  0.754  after adding  14
	Accuracy:  0.762  after adding  16
	Accuracy:  0.768  after adding  17
	Accuracy:  0.772  after adding  18
	Accuracy:  0.778  after adding  20
	Accuracy:  0.796  after adding  23
	Accuracy:  0.786  after adding  26
	Accuracy:  0.784  after adding  28
	Accuracy:  0.77  after adding  29
	Accuracy:  0.782  after adding  30
	Accuracy:  0.774  after adding  31
	Accuracy:  0.78  after adding  32
	Accuracy:  0.77  after adding  33
	Accuracy:  0.78  after adding  34
	Accuracy:  0.758  after adding  35
	Accuracy:  0.756  after adding  36
	A

 41%|██████████████▍                    | 33/80 [26:12:54<31:51:02, 2439.63s/it]

	Accuracy:  0.784  after adding  79
Attention!! Accuracy is decreasing!
Best feature:  47  with accuracy:  0.798
Feature list:  [10, 63, 71, 13, 41, 25, 22, 1, 8, 19, 77, 24, 45, 6, 27, 39, 49, 75, 66, 15, 38, 74, 21, 51, 56, 7, 50, 55, 80, 4, 37, 5, 47]

	Accuracy:  0.768  after adding  2
	Accuracy:  0.766  after adding  3
	Accuracy:  0.772  after adding  9
	Accuracy:  0.78  after adding  11
	Accuracy:  0.768  after adding  12
	Accuracy:  0.752  after adding  14
	Accuracy:  0.766  after adding  16
	Accuracy:  0.76  after adding  17
	Accuracy:  0.77  after adding  18
	Accuracy:  0.766  after adding  20
	Accuracy:  0.764  after adding  23
	Accuracy:  0.772  after adding  26
	Accuracy:  0.768  after adding  28
	Accuracy:  0.778  after adding  29
	Accuracy:  0.776  after adding  30
	Accuracy:  0.766  after adding  31
	Accuracy:  0.782  after adding  32
	Accuracy:  0.77  after adding  33
	Accuracy:  0.77  after adding  34
	Accuracy:  0.762  after adding  35
	Accuracy:  0.768  after adding 

 42%|██████████████                   | 34/80 [34:45:22<139:41:10, 10931.96s/it]

	Accuracy:  0.782  after adding  79
Attention!! Accuracy is decreasing!
Best feature:  32  with accuracy:  0.782
Feature list:  [10, 63, 71, 13, 41, 25, 22, 1, 8, 19, 77, 24, 45, 6, 27, 39, 49, 75, 66, 15, 38, 74, 21, 51, 56, 7, 50, 55, 80, 4, 37, 5, 47, 32]

	Accuracy:  0.776  after adding  2
	Accuracy:  0.772  after adding  3
	Accuracy:  0.76  after adding  9
	Accuracy:  0.774  after adding  11
	Accuracy:  0.76  after adding  12
	Accuracy:  0.75  after adding  14
	Accuracy:  0.77  after adding  16
